In [119]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import copy
import os
print(os.listdir("../input"))
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
# Any results you write to the current directory are saved as output.

['creditcard.csv']


In [120]:
df = pd.read_csv("../input/creditcard.csv")

In [121]:
df.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [122]:
print('总交易次数为:', len(df))
print('正常交易次数为：', len(df[df['Class']==0]))
print('欺诈交易次数为：', len(df[df['Class']==1]))

总交易次数为: 284807
正常交易次数为： 284315
欺诈交易次数为： 492


分析数据集，交易数据是经过脱敏之后，用28维向量描述的数据。数据集包含了284807条交易数据，其中正常交易次数为284315，欺诈交易次数为492。数据极不平衡。

## 特征提取
1. 标准化
数据集的V1-V28特征已经归一化处理了，仅剩Amount字段是原始的交易数据，接下来首先将该字段进行标准化处理。

In [123]:
df1 = copy.deepcopy(df)
df['normAmount'] = preprocessing.StandardScaler().fit_transform(df['Amount'].values.reshape(-1, 1))

In [126]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class,normAmount
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0,0.244964
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0,-0.342475
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0,1.160686
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0,0.140534
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0,-0.073403


删除Time、Amount两个条目

In [127]:
df = df.drop(['Time', 'Amount'], axis=1)

将标签赋值给y

In [128]:
y = df['Class']

In [129]:
features = df.drop(['Class'], axis=1).columns

In [130]:
x = df[features]

划分训练集和测试集，其中训练集占60%，测试集占40%

In [131]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)

## 训练朴素贝叶斯模型

In [132]:
gnb = GaussianNB()
gnb.fit(x_train, y_train)
y_pred = gnb.predict(x_test)

评估模型准确率

In [133]:
print('metrics.accuracy_score:', metrics.accuracy_score(y_test, y_pred))
print('metrics.confusion_score:', metrics.confusion_matrix(y_test, y_pred))
print('metrics.precision_score:', metrics.precision_score(y_test, y_pred))
print('metrics.recall_score:', metrics.recall_score(y_test, y_pred))
print('metrics.f1_score:', metrics.f1_score(y_test, y_pred))

metrics.accuracy_score: 0.9784415789612282
metrics.confusion_score: [[111321   2421]
 [    35    146]]
metrics.precision_score: 0.05687573042462018
metrics.recall_score: 0.8066298342541437
metrics.f1_score: 0.10625909752547306


由于这里我们的任务是检测欺诈交易，所以我们在评估模型效果时主要参考精度precision和召回率recall。
可以看到模型的准确率为5.69%，召回率为80.66%。

## 降采样
由于这里正常交易数量和欺诈交易数量比严重失衡，所以需要对数据做降采样处理，即从数据量多的一方随机选取与较少一方相当数量的数据。这里我们从正常交易数据中随机取492条数据。

In [134]:
number_fraud = len(df[df.Class == 1])
fraud_index = np.array(df[df.Class == 1].index)

In [135]:
normal_index = df[df.Class == 0].index
random_choice_index = np.random.choice(normal_index, size=number_fraud, replace=False)

将降采样后的数据与欺诈交易数据重新组合成新的样本集合

In [136]:
x_index = np.concatenate([fraud_index, random_choice_index])
df = df.drop(['Class'], axis=1)
x = df.iloc[x_index, :]
y = [1] * number_fraud + [0] * number_fraud

In [137]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)

In [138]:
gnb = GaussianNB()
gnb.fit(x_train, y_train)
y_pred = gnb.predict(x_test)

In [139]:
print('metrics.accuracy_score:', metrics.accuracy_score(y_test, y_pred))
print('metrics.confusion_score:', metrics.confusion_matrix(y_test, y_pred))
print('metrics.precision_score:', metrics.precision_score(y_test, y_pred))
print('metrics.recall_score:', metrics.recall_score(y_test, y_pred))
print('metrics.f1_score:', metrics.f1_score(y_test, y_pred))

metrics.accuracy_score: 0.8984771573604061
metrics.confusion_score: [[195   9]
 [ 31 159]]
metrics.precision_score: 0.9464285714285714
metrics.recall_score: 0.8368421052631579
metrics.f1_score: 0.8882681564245811


经过降采样后，模型的精度提升到了94.64%，召回率为83.68%

## 过采样
除了降采样，还可以采用过采样来处理数据不平衡问题。即在数量较少样本的基础上生成新样本，最常用的生成算法是Smote。  
Smote算法的思想是先随机选定n个少类的样本，再找出最靠近它的m个少类样本，接着任选最临近的m个少类样本中的任意一点，在这两点上任选一点，这个点就是新增的样本。

In [140]:
from imblearn.over_sampling import SMOTE

In [146]:
features = df1.drop(['Class'], axis=1).columns
x = df1[features]
y = df1['Class']

In [147]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)

In [148]:
print(pd.value_counts(y_train))

0    170585
1       299
Name: Class, dtype: int64


未作处理直接进行train_test_split划分，可以看到样本分布的确不均衡。  
接下来我们使用Smote算法对训练数据进行过采样。  

In [149]:
os = SMOTE(random_state=0)
x_train_1, y_train_1 = os.fit_sample(x_train, y_train)
print('Smote data')
print(pd.value_counts(y_train_1))

Smote data
1    170585
0    170585
dtype: int64


Smote算法处理后看到样本达到了均衡，接下来训练模型看看效果如何！

In [150]:
gnb = GaussianNB()
gnb.fit(x_train_1, y_train_1)
y_pred = gnb.predict(x_test)

In [151]:
print('metrics.accuracy_score:', metrics.accuracy_score(y_test, y_pred))
print('metrics.confusion_score:', metrics.confusion_matrix(y_test, y_pred))
print('metrics.precision_score:', metrics.precision_score(y_test, y_pred))
print('metrics.recall_score:', metrics.recall_score(y_test, y_pred))
print('metrics.f1_score:', metrics.f1_score(y_test, y_pred))

metrics.accuracy_score: 0.9916171449136698
metrics.confusion_score: [[112819    911]
 [    44    149]]
metrics.precision_score: 0.14056603773584905
metrics.recall_score: 0.772020725388601
metrics.f1_score: 0.237829209896249


模型精度14.06%，召回率77.20%  
可以看到Smote之后精度要比未经过均衡处理时的效果好，但是由于样本数据是生成的，和实际数据还是有一定区别，而且训练数据量要比降采样时多得多，所以在结果上看到要比降采样的结果"差"一些。